In [1]:
# https://www.kaggle.com/c/diabetic-retinopathy-detection/discussion/242755
# https://github.com/aladdinpersson/Machine-Learning-Collection/tree/master/ML/Kaggles/DiabeticRetinopathy
# https://www.youtube.com/watch?v=YxQYvhap3kE&list=PLhhyoLH6IjfxkVb3Yx4mjLdwMWP5S_Hct

import torch
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm

In [3]:
# Configurations
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 5e-4
BATCH_SIZE = 128
NUM_EPOCHS = 120
NUM_WORKERS = 6
PIN_MEMORY = True
LOAD_MODEL = True
SAVE_MODEL = True
CHECKPOINT_FILE = "diabetic_retinopathy_detection.pth.tar"

# Data augmentation for images
train_transform = A.Compose([
    A.Resize(width=150, height=150),
    A.RandomCrop(width=120, height=120),
    A.Normalize(mean=(0.3199, 0.2240, 0.1609),
                std=(0.3020, 0.2183, 0.1741),
                max_pixel_value=255.0),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(width=150, height=150),
    A.Normalize(mean=(0.3199, 0.2240, 0.1609),
                std=(0.3020, 0.2183, 0.1741),
                max_pixel_value=255.0),
    ToTensorV2()
])

In [12]:
img_file_paths = os.listdir("../data/kaggle-diabetic-retinopathy-detection/train")
print(len(img_file_paths))
print(img_file_paths[:5])
img_file_paths[0].replace(".jpeg", "")

35126
['10003_left.jpeg', '10003_right.jpeg', '10007_left.jpeg', '10007_right.jpeg', '10009_left.jpeg']


'10003_left'

In [10]:
df_train = pd.read_csv("../data/kaggle-diabetic-retinopathy-detection/trainLabels.csv")
print(df_train.head())
im, le = df_train.iloc[0]
print(im, le)

      image  level
0   10_left      0
1  10_right      0
2   13_left      0
3  13_right      0
4   15_left      1
10_left 0


In [3]:
import glob


# 对高分辨率图片进行缩放
def resize_images(input_folder, output_folder, target_size=(650, 650)):
    # 确保输出文件夹存在
    os.makedirs(output_folder, exist_ok=True)

    # 获取支持的图片格式
    img_extensions = ["*.jpg", "*.jpeg", "*.png", "*.bmp", "*.tiff"]
    img_file_paths = []
    for ext in img_extensions:
        img_file_path = os.path.join(input_folder, '**', ext)
        img_file_paths.extend(glob.glob(img_file_path, recursive=True))
    print(f"找到 {len(img_file_paths)} 张图片需要处理")

    # 遍历处理所有图片文件
    for img_file_path in tqdm(img_file_paths, desc="Resizing images"):
        try:
            # 获取相对路径
            relative_path = os.path.relpath(img_file_path, input_folder)
            output_file_path = os.path.join(output_folder, relative_path)

            # 确保输出文件夹存在
            os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

            # 打开图片
            with Image.open(img_file_path) as img:
                # 使用高质量的LANCZOS重采样算法
                resized_img = img.resize(target_size, Image.LANCZOS)
                # 保存图片
                resized_img.save(output_file_path, quality=95)
        except Exception as e:
            print(f"处理图片 {img_file_path} 时出错: {str(e)}")

    print("图片调整完成!")


samples = r'../data/kaggle-diabetic-retinopathy-detection/sample'
# 创建输出文件夹
samples_output = os.path.join(samples, 'resized')
resize_images(samples, samples_output, target_size=(150, 150))
print(f"所有图片处理完成！调整后的图片已保存至: {samples_output}")

找到 10 张图片需要处理


Resizing images: 100%|██████████| 10/10 [00:00<00:00, 10.72it/s]

图片调整完成!
所有图片处理完成！调整后的图片已保存至: ../data/kaggle-diabetic-retinopathy-detection/sample\resized


In [4]:
train_images_folder = r"../data/kaggle-diabetic-retinopathy-detection/train"
resized_train_images_folder = os.path.join(train_images_folder, 'images_resized_650')
resize_images(train_images_folder, resized_train_images_folder, target_size=(650, 650))
print(f"所有图片处理完成！调整后的图片已保存至: {resized_train_images_folder}")

找到 35126 张图片需要处理


Resizing images: 100%|██████████| 35126/35126 [48:41<00:00, 12.02it/s] 

图片调整完成!
所有图片处理完成！调整后的图片已保存至: ../data/kaggle-diabetic-retinopathy-detection/train\images_resized_650


In [ ]:
class DRDataset(Dataset):
    def __init__(self,
                 image_folder: str,
                 csv_file_path: str,
                 train=True,
                 transform=None):
        super().__init__()
        self.df = pd.read_csv(csv_file_path)
        self.image_folder = image_folder
        self.image_file_names = os.listdir(self.image_folder)  # ['10003_left.jpeg', '10003_right.jpeg']
        self.train = train
        self.transform = transform

    def __len__(self):
        return len(self.df) if self.train else len(self.image_file_names)

    def __getitem__(self, idx):
        if self.train:
            image_name, label = self.df.iloc[idx]
        else:
            # return -1 for test label
            label = -1
            image_file_name = self.image_file_names[idx]
            image_name = image_file_name.replace(".jpeg", "")
        image_file_path = os.path.join(self.image_folder, image_name + ".jpeg")
        image = np.array(Image.open(image_file_path))

        if self.transform:
            image = self.transform(image)["image"]

        return image, label, image_name


dataset = DRDataset(image_folder=r"../data/kaggle-diabetic-retinopathy-detection/train",
                    csv_file_path=r"../data/kaggle-diabetic-retinopathy-detection/trainLabels.csv",
                    transform=train_transform)

loader = DataLoader(dataset, batch_size=32, num_workers=2, shuffle=True, pin_memory=True)
for X, y, name in tqdm(loader):
    print(X.shape, y.shape, name)
    import sys

    sys.exit()

  0%|          | 0/1098 [00:00<?, ?it/s]

In [2]:
51786.5 / 1886.05

27.457649585111742

In [2]:
from utils import trim, fast_resize_images
from PIL import Image

image_original = Image.open(r"../data/kaggle-diabetic-retinopathy-detection/train/10003_left.jpeg")
image_original.show()
image_trimmed = trim(image_original)
image_trimmed.show()

In [7]:
from utils.preprocess_images import fast_resize_images
import os
train_images_folder = r"../data/kaggle-diabetic-retinopathy-detection/train"
resized_train_images_folder = os.path.join(train_images_folder, 'images_resized_150')
fast_resize_images(train_images_folder, resized_train_images_folder, output_size=(150, 150))

找到 70252 张图片需要处理
使用 24 个CPU核心进行处理


100%|██████████| 70252/70252 [22:10<00:00, 52.79it/s]

处理完成! 结果保存在 ../data/kaggle-diabetic-retinopathy-detection/train\images_resized_150
